![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)

Find this notebook in https://colab.research.google.com/github/ricardokleinklein/NLP_GenMods/blob/main/fake_news.ipynb

# Modelos Generativos

## Detección de Fake News - NLP

Creado por *Ricardo Kleinlein* para [Saturdays.AI](https://saturdays.ai/).

Disponible bajo una licencia [Creative Commons](https://creativecommons.org/licenses/by/4.0/).

---

## Sobre el uso de Jupyter Notebooks

Este notebook ha sido implementado en Python, pero para su ejecución no es
necesario conocer el lenguaje en profundidad. Solamente se debe ejecutar cada
una de las celdas, teniendo en cuenta que hay que ejecutar una celda a la vez
y secuencialmente, tal y como figuran en orden de aparición.

Para ejecutar cada celda pulse en el botón ▶ en la esquina superior izquierda
de cada celda. Mientras se esté ejecutando ese fragmento de código,
el botón estará girando. En caso de querer detener dicha ejecución, pulse
nuevamente sobre este botón mientras gira y la ejecución se detendrá. En caso
de que la celda tenga alguna salida (texto, gráficos, etc) será mostrada
justo después de esta y antes de mostrar la siguiente celda. El notebook
estará guiado con todas las explicaciones necesarias, además irá acompañado
por comentarios en el código para facilitar su lectura.

En caso de tener alguna duda, anótela. Dedicaremos un tiempo a plantear y
resolver la mayoría delas dudas que puedan aparecer.


## Objetivo del notebook

Implementar, entrenar y evaluar un modelo de predicción de noticias falsas
relacionadas con la pandemia de la COVID-19 en inglés.

Dicho modelo estará basado en una red neuronal profunda recurrente que
observará vectores de palabra preentrenados.


## Importamos las librerías

In [7]:
import matplotlib.pyplot as plt
import pandas
import torch

## Accedemos a la base de datos

Si la celda inferior fallara, podemos subir el archivo manualmente a Google
Colab.

In [33]:
df_train = pandas.read_csv('./assets/news_train.csv')
df_test = pandas.read_csv('./assets/news_test.csv')

def news_iterator(tweets, labels):
    for tweet, label in zip(tweets, labels):
        yield (label, tweet)

train_iter = news_iterator(df['tweet'].values, df['label'].values)

## Qué aspecto tienen los datos?

Al ser una clasificación binaria (real / fake), solo van a existir 2
categorías.

In [34]:
print(df.head(5))
df['label'].describe()

   id                                              tweet label
0   1  The CDC currently reports 99031 deaths. In gen...  real
1   2  States reported 1121 deaths a small rise from ...  real
2   3  Politically Correct Woman (Almost) Uses Pandem...  fake
3   4  #IndiaFightsCorona: We have 1524 #COVID testin...  real
4   5  Populous states can generate large case counts...  real


count     6420
unique       2
top       real
freq      3360
Name: label, dtype: object

Como vemos, es una tarea en la que las etiquetas están bastante equilibradas
 (52% de verdaderas, 48% de noticias falsas), convirtiendo el problema en un
  caso estándar susceptible de ser atacado mediante machine learning.

## Preprocesado

Como hemos comentado en otro notebook, un paso fundamental consiste en
tokenizar nuestras palabras y construir con ello un vocabulario de nuestra
tarea.

In [25]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

Con este procesado pasamos de palabras a una lista de índices:

In [26]:
vocab(['here', 'is', 'an', 'instance'])

[142, 10, 65, 17852]

Vamos ahora a preparar de terminar el pipeline de entrada de texto, junto
con la entrada del label de cada muestra.

In [35]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: 0 if x == 'fake' else 1

Esto es en realidad una look-up table definida por la manera de construir
nuestro vocabulario. Igualmente, convertimos las etiquetas en un índice, en
lugar de su forma original.

In [36]:
text_pipeline('here is a new instance')
label_pipeline('fake')

0

## Generar batches e iterador

In [37]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

## Definimos el modelo a usar

In [38]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [39]:
model = TextClassificationModel(vocab_size=len(vocab), embed_dim=64,
                                num_class=2)

## Definir la función de entrenamiento y evaluación

In [40]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

